# 태그 읽기 프로그램

ID3v2.3 정보는 헤더에 작성된 사이즈 만큼 순차적으로 작성이 되어있기 때문에 어떤 부분은 읽고 어떤 부분은 안읽고 하는 선택적으로 원하는 내용만 읽어오게되면 파일포인터가 꼬이는 경우가 생기기 때문에 어쨌든 해당 파일에 정의된 내용은 모두 읽어와야 파일포인터가 꼬이지 않고 데이터를 문제 없이 읽어올 수 있습니다.

MP3 파일에서 대략적으로 어떻게 ID3 태그 정보를 읽어오는지 테스트 했으니 해당 내용을 기반으로 좀 더 완성도 있게 MP3 파서 프로그램을 테스트 해보도록 하겠습니다.

In [3]:
from PIL import Image
import io
from io import BytesIO
import time

TIF = ["TALB", "TBPM", "TCOM", "TCON", "TCOP", "TDAT", "TDLY", "TENC", "TEXT", 
        "TFLT", "TIME", "TIT1", "TIT2", "TIT3", "TKEY", "TLAN", "TLEN", "TMED", 
        "TOAL", "TOFN", "TOLY", "TOPE", "TORY", "TOWN", "TPE1", "TPE2", "TPE3",
        "TPE4", "TPOS", "TPUB", "TRCK", "TRDA", "TRSN", "TRSO", "TSIZ", "TSRC",
        "TSSE", "TYER"]

class MP3Parser(object):
    def __init__(self, filename):
        self.filename = filename
        self.ID3_TAG = None
        self.ID3_VERSION = None
        self.ID3_FLAGS = None
        self.ID3_SIZE = None
        self.ID3_BYTES = None
        self.ID3_TAGS = []

    def __byte_to_int(self, data):
        return int(data.hex(), 16)

    def __byte_to_str(self, data):
        return data.decode("utf-8")

    def __slice_null_data(self, bio):
        _data = bytearray()
        while True:
            _null = bio.read(1)
            if _null == b"\x00" or _null == b"":
                break
            _data.append(self.__byte_to_int(_null))
        return _data
    
    def __decode_header_size(self, str_hex):
        n = 2
        slice_hex = [str_hex[i:i+n] for i in range(0, len(str_hex), n)]
        results = ""
        for h in slice_hex:
            results += "{0:b}".format(int(h, 16)).zfill(7)
        return int(results, 2)

    def __parse_header(self, header):
        _bio = BytesIO(header)
        _fid = self.__byte_to_str(_bio.read(4))
        _size = self.__byte_to_int(_bio.read(4))
        _flags = _bio.read(2)
        return (_fid, _size, _flags)

    def __get_text_information(self, fid, data):
        _bio = BytesIO(data)
        _encoding = _bio.read(1)
        if fid in TIF:
            decoding = "UTF-16" if _encoding == b"\x01" else "cp949"
            _byte_value = _bio.read()
            _decoding_value = _byte_value.decode(decoding).split("\x00")[0].strip()
            return {
                "fid": fid,
                "value": _decoding_value
            }
        else:
            return None

    def __decoding_text(self, encoding, bio):
        decoding = "UTF-16" if encoding == b"\x01" else "cp949"
        _byte_value = bio.read()
        _decoding_value = _byte_value.decode(decoding).split("\x00")[0].strip()
        return _decoding_value

    def __get_lyrics(self, data):
        _bio = BytesIO(data)
        _encoding = _bio.read(1)
        _lang = _bio.read(3)
        _descriptor = self.__slice_null_data(_bio)
        _bio.read(1)
        _data_lyrics = self.__decoding_text(_encoding, _bio)
        return {
            "fid": "USLT",
            "lang": _lang,
            "descriptor": _descriptor,
            "lyrics": _data_lyrics
        }

    def __get_picture(self, data):
        _bio = BytesIO(data)
        _encoding = _bio.read(1)
        _mime_type = self.__slice_null_data(_bio)
        _pic_type = _bio.read(1)
        _description = self.__slice_null_data(_bio)
        _image_binary = _bio.read()
        image = Image.open(io.BytesIO(_image_binary))
        return {
            "fid": "APIC",
            "mime_type": _mime_type,
            "pic_type": _pic_type,
            "description": _description,
            "binary": _image_binary
        }
        
    def parsing(self):
        mp3 = open(self.filename, "rb")
        header_size = 10
        header_data = mp3.read(header_size)
        id3_header = BytesIO(header_data)
        self.ID3_TAG = id3_header.read(3)
        self.ID3_VERSION = id3_header.read(2)
        self.ID3_FLAGS = id3_header.read(1)
        self.ID3_SIZE = id3_header.read(4)
        self.ID3_BYTES = self.__decode_header_size(self.ID3_SIZE.hex())
        ID3_DATA = BytesIO(mp3.read(self.ID3_BYTES))
        
        while True:
            _fid, _fsize, _fflags = self.__parse_header(ID3_DATA.read(10))
            _fdata = ID3_DATA.read(_fsize)
            r = None
            if ID3_DATA.tell() == self.ID3_BYTES or _fdata == b"":
                break
            if _fid == "USLT":
                r = self.__get_lyrics(_fdata)
            elif _fid == "APIC":
                r = self.__get_picture(_fdata)
            else:
                r = self.__get_text_information(_fid, _fdata)
            if r is not None:
                self.ID3_TAGS.append(r)

    def print_all(self):
        print(self.ID3_TAGS)

    def show_cover_image(self):
        image = next((item for item in self.ID3_TAGS if item['fid'] == "APIC"), None)
        if image is not None:
            image = Image.open(io.BytesIO(image.get("binary")))
            image.show()

filename = "bensound-newdawn.mp3"
mp3parser = MP3Parser(filename)
mp3parser.parsing()
mp3parser.print_all()
mp3parser.show_cover_image()

[{'fid': 'TALB', 'value': 'ROYALTY FREE MUSIC'}, {'fid': 'TPE1', 'value': 'Benjamin Tissot'}, {'fid': 'TPE2', 'value': 'Benjamin Tissot (also known as Bensound)'}, {'fid': 'TCOM', 'value': 'Benjamin Tissot (also known as Bensound)'}, {'fid': 'TCOP', 'value': 'https://www.bensound.com'}, {'fid': 'TCON', 'value': 'cinematic'}, {'fid': 'TIT2', 'value': 'NEW DAWN'}, {'fid': 'USLT', 'lang': b'kor', 'descriptor': bytearray(b'\xff\xfe'), 'lyrics': '테스트 가사입니다.\r\n이곳에 가사 데이터가 들어갑니다.\r\n가사 데이터에는 개행 문자가 들어갈 수 있습니다.'}, {'fid': 'TYER', 'value': '2021'}, {'fid': 'APIC', 'mime_type': bytearray(b'image/jpeg'), 'pic_type': b'\x03', 'description': bytearray(b''), 'binary': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x005\x00\x00\xff\xee\x00!Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x10\x03\x03\x06\t\x00\x00\x18\x19\x00\x00$\xd4\x00\x001\x8a\xff\xdb\x00\x84\x00\x08\x05\x05\x05\x06\x05\x08\x06\x06\x08\x0b\x07\x06\x07\x0b\r\t\x08\x08\t\r\x0f\x0

위 코드는 지금까지 배운 내용을 토대로 클래스화 시켜서 작성해본 MP3 ID3 태그 파서 프로그램입니다. 현재 샘플로 사용하고 있는 MP3 파일의 기본적인 태그는 포함되어있으나 ID3v2.3 의 모든 태그를 포함하지 않기 때문에 다른 MP3 파일을 테스트 했을때 만약 존재하지 않는 태그가 있다면 파일포인터의 오류가 발생할 수 있습니다. 